# [Reference](https://www.kaggle.com/code/kaggle5daysofai/day-2a-agent-tools/notebook)

In [1]:
import os
from kaggle_secrets import UserSecretsClient

GEMINI_API_KEY = UserSecretsClient().get_secret("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [2]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext, FunctionTool
from google.adk.code_executors import BuiltInCodeExecutor

In [3]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        if(
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                     print(
                         "Generated Python Response >> ", response_code["result"]
                     )
        else:
            print("None")


                    
    

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

In [5]:
def get_fee_for_payment_method(pay_method: str) -> dict:
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(pay_method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{pay_method}' not found",
        }

In [6]:
def get_exchange_rate(base_cur:str, target_cur:str) -> dict:
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    base = base_cur.lower()
    target = target_cur.lower()
    
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }

In [7]:
cur_agent = LlmAgent(
    name = "cur_agent",
    model = Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    instruction = """You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools = [FunctionTool(get_fee_for_payment_method), FunctionTool(get_exchange_rate)]
)

In [8]:
async with InMemoryRunner(agent=cur_agent) as runner:
    response = await runner.run_debug(
        """
        Convert 10,000 USD to INR using a Bank Transfer. Show me the precise calculation.
        """
    )


 ### Created new session: debug_session_id

User > 
        Convert 10,000 USD to INR using a Bank Transfer. Show me the precise calculation.
        


cur_agent > The final converted amount is 827442.0 INR.

Here's the precise calculation:

1. The fee for a Bank Transfer is 1% of the original amount, which is 100.00 USD.
2. The amount remaining after the fee is 9900.00 USD.
3. The exchange rate from USD to INR is 83.58.
4. The final converted amount is 9900.00 USD * 83.58 = 827442.0 INR.



In [9]:
print(response[0].content.parts[0])

function_call=FunctionCall(
  args={
    'pay_method': 'Bank Transfer'
  },
  id='adk-d1a156a7-00de-4920-8645-5590313ac820',
  name='get_fee_for_payment_method'
) code_execution_result=None executable_code=None file_data=None function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None


In [10]:
cal_agent = LlmAgent(
    name = "cal_agent",
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction="""
    You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
    """,

    code_executor = BuiltInCodeExecutor()
)

In [11]:
root_agent = LlmAgent(
    name="root_agent",
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = """
    You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools = [FunctionTool(get_fee_for_payment_method), FunctionTool(get_exchange_rate), AgentTool(agent=cal_agent)]
)

In [12]:
async with InMemoryRunner(agent=root_agent) as runner:
    response = await runner.run_debug(
        """
         Convert 10,000 USD to INR using a Bank Transfer. Show me the precise calculation.
         """
    )


 ### Created new session: debug_session_id

User > 
         Convert 10,000 USD to INR using a Bank Transfer. Show me the precise calculation.
         


root_agent > The final converted amount is 834,164.2 INR.

Here's a breakdown of the calculation:
*   The transaction fee for a Bank Transfer is 0.01%, which amounts to 1 USD.
*   After deducting the fee, the remaining amount is 9,999 USD.
*   This remaining amount was converted to INR at an exchange rate of 1 USD = 83.58 INR, resulting in the final amount of 834,164.2 INR.


In [20]:
# print(show_python_code_and_result(response))
print(response[3].content.parts[0].function_response.response)

{'status': 'success', 'rate': 83.58}


In [14]:
show_python_code_and_result(response)

None
None
None
Generated Python Code >>  I need to calculate the final amount in INR after converting 10,000 USD to INR, considering a fee of 0.01% and an exchange rate of 83.58.

Here's the plan:
1. Calculate the fee amount in USD.
2. Subtract the fee from the initial USD amount to get the net USD amount.
3. Convert the net USD amount to INR using the given exchange rate.
4. Print the final INR amount.


```python
initial_usd = 10000
fee_percentage = 0.0001  # 0.01%
exchange_rate = 83.58

# Calculate the fee amount in USD
fee_usd = initial_usd * fee_percentage

# Calculate the net USD amount after deducting the fee
net_usd = initial_usd - fee_usd

# Convert the net USD amount to INR
final_inr = net_usd * exchange_rate

print(f'{final_inr=}')
```
```json
{
"": "import sympy\n\ninitial_usd = 10000\nfee_percentage = 0.0001  # 0.01%\nexchange_rate = 83.58\n\n# Calculate the fee amount in USD\nfee_usd = initial_usd * fee_percentage\n\n# Calculate the net USD amount after deducting the fee\